In [1]:
# for PyCharm
import os

import numpy as np

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [2]:

from pathlib import Path
import shutil

from src.data.merger import Merger

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# datasets = Merger().get_formatted()
# for key, value in datasets.items():
#     print(value.shape)
# same_columns = set.intersection(*[set(datasets[key].columns) for key in datasets])
# print(same_columns)
# Merger.check_extra_symbols(datasets, columns=["brand", "title"])

In [4]:
merged_essentials, merged_dataset = Merger().get_merged_dataset(Path("data") / "merged" / "metadata")
print(merged_essentials.shape)
print(merged_dataset.shape)

superkicks columns Index(['brand', 'title', 'price', 'collection_name', 'url', 'images_path',
       'website', 'pricecurrency', 'title_without_color', 'color',
       'title_merge', 'brand_merge'],
      dtype='object') size (1141, 12)
sneakerbaas columns Index(['brand', 'pricecurrency', 'price', 'title', 'collection_name', 'url',
       'images_path', 'website', 'title_without_color', 'color', 'title_merge',
       'brand_merge'],
      dtype='object') size (926, 12)
footshop columns Index(['brand', 'title', 'color', 'pricecurrency', 'price', 'collection_name',
       'url', 'images_path', 'website', 'title_without_color', 'title_merge',
       'brand_merge'],
      dtype='object') size (4852, 12)
{'superkicks': ["'", '"', '+', '(', ')', '.', '#'], 'sneakerbaas': ['"', "'", '.', '(', ')', '+'], 'footshop': ["'", '"', '.', '®', '*', '™', '+', '(', ')', '?', ',', ':', '.', '®', "'", '*']}
6919 -> 2484
(2484, 3)
(2484, 13)


In [5]:
# images = temp["images_path"].values
# for image_list in images:
#     for image in image_list:
#         if not isinstance(image, str):
#             print(image_list)
#             print(type(image))
# import numpy as np
# len(set(np.hstack(images)))

In [6]:
# counter = 0
# 
# for source_path in set(np.hstack(images).tolist()):
#     source_path = Path(source_path)
# 
#     if source_path.is_dir():
#         for source_file in source_path.glob('*'):
#             counter += 1
# 
#     elif source_path.is_file():
#         counter += 1
#     else:
#         print(source_path)
# 
# counter

In [11]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)

path = Path("data") / "merged" / "images"
merged_essentials.apply(lambda x: copy_files_to_destination(x["images_path"], path / "by-brands" / x["brand_merge"]),
                        axis=1)

0          4
1         12
2         13
3         23
4         24
        ... 
2479    7676
2480    7688
2481    7694
2482    7700
2483    7706
Length: 2484, dtype: int64

In [12]:
merged_essentials.apply(lambda x: copy_files_to_destination(x["images_path"], path / "by-models" / x["title_merge"]),
                        axis=1)

0        4
1        7
2        0
3        9
4        0
        ..
2479    11
2480    11
2481     5
2482     5
2483     5
Length: 2484, dtype: int64

In [10]:
from src.data.storage import StorageProcessor
from src.data.local import LocalStorage

processor = StorageProcessor(LocalStorage())
test = ['data/raw/images/kickscrew/Air Jordan/air-jordan-1-retro-high-og-rebellionaire-555088-036-right-side-img.05705',
        'data/raw/images/kickscrew/Air Jordan/air-jordan-1-retro-high-og-rebellionaire-555088-036-left-side-img.05705',
        'data/raw/images/kickscrew/Air Jordan/air-jordan-1-retro-high-og-rebellionaire-555088-036-front-both-img.05705',
        'data/raw/images/kickscrew/Air Jordan/air-jordan-1-high-og-hand-crafted-dh3097-001-right-side-img.10771',
        'data/raw/images/kickscrew/Air Jordan/air-jordan-1-high-og-hand-crafted-dh3097-001-left-side-img.10771',
        'data/raw/images/kickscrew/Air Jordan/air-jordan-1-high-og-hand-crafted-dh3097-001-front-both-img.10771',
        "data/raw/images/superkicks/women-sneakers/vans/wmns-ashetr-multi",
        "data/raw/images/superkicks/women-sneakers/vans/utility-pop-old-skool-yellow"]


def copy_files_to_destination(source_list, destination):
    if len(source_list) == 0:
        return 0

    destination_path = Path(destination)
    destination_path.mkdir(parents=True, exist_ok=True)

    counter = processor.get_max_filename(str(destination_path))

    for source_path in source_list:
        source_path = Path(source_path)

        if source_path.is_dir():
            for source_file in source_path.glob('*'):
                counter += 1
                destination_file = destination_path / f"{counter}{source_file.suffix}"
                shutil.copy2(source_file, destination_file)


        elif source_path.is_file():
            counter += 1
            destination_file = destination_path / f"{counter}{source_path.suffix}"
            shutil.copy2(source_path, destination_file)

    return counter

# copy_files_to_destination(test, "data/merged/images/test")

In [ ]:
# import numpy as np
# flat_colors = np.concatenate(merged["color"].values).tolist()
# flat_colors = np.concatenate([item if isinstance(item, list) else [item] for item in flat_colors]).tolist()
# flat_colors
# out = set()
# for color in flat_colors:
#     out.add(Merger.format_merge(color))
# out = list(out)
# out.sort(key=len, reverse=True)
# out
# with open("notebooks/merger/color_words.txt", "a") as f:
#     f.write('\n'.join(out))

## Seara heavymetal experiments:

In [ ]:
# !pip install thefuzz -q

In [ ]:
# from thefuzz import fuzz
# 
# 
# def similarity(string1, string2):
#     return fuzz.token_sort_ratio(string1, string2)
# 
# 
# title = "vans - ua og authentic lx yellow"
# 
# similarities = []
# for text in titles:
#     similarities.append((text, similarity(title, text)))
# 
# similarities.sort(key=lambda x: x[1])
# similarities[-10:]

In [ ]:
# эта тема лучше всего находит похожие